# Resource Validation

Created for air-gapped clouds, this notebook validates whether the VBD resources in this solution are available and configured in the current subscription.  

### Python Prerequisite Libraries for Validation

The following libraries may need to be installed in your kernel to validate the Azure session.  If you do NOT have a connection to the general web, you will need to find a secure means to load these packages on your own Python kernel (using Terminal for your compute instance is the recommended way).

In [2]:
# Uncomment to install directly from the Jupyter notebook (this method assumes the ability to reach the regular web)
# !pip install azure-mgmt-resource azure-mgmt-search azure-identity azure.mgmt.cognitiveservices

     |████████████████████████████████| 144 kB 4.5 MB/s eta 0:00:01
ERROR: azure-cli 2.49.0 has requirement azure-keyvault-keys==4.8.0b2, but you'll have azure-keyvault-keys 4.8.0 which is incompatible.
ERROR: azure-cli 2.49.0 has requirement azure-mgmt-cognitiveservices~=13.3.0, but you'll have azure-mgmt-cognitiveservices 13.5.0 which is incompatible.
ERROR: azure-cli 2.49.0 has requirement azure-mgmt-keyvault==10.2.0, but you'll have azure-mgmt-keyvault 10.2.1 which is incompatible.
ERROR: azure-cli 2.49.0 has requirement azure-mgmt-resource==22.0.0, but you'll have azure-mgmt-resource 21.1.0b1 which is incompatible.


### Python Package Load

These packages will be used to validate the resources in this Azure subscription.

In [8]:
from azureml.core import Workspace
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.search import SearchManagementClient
from azure.identity import DefaultAzureCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
import pkg_resources

import requests
import os
import json
from dotenv import load_dotenv

try:
    load_dotenv(dotenv_path="credentials.env")
    print("Successfully loaded environment variables from credentials.env")
    
except Exception as e:
    print(f"An error occurred while loading environment variables from credentials.env: {e}")

Successfully loaded environment variables from credentials.env


### Validation: Requirements File

This VBD has a "./common/requirements.txt" file whose contents may change in future VBD releases.  The following code checks that the current version has its libraries installed in the kernel running in Jupyter.

In [4]:
def validate_requirements(requirements_path):
    # Read requirements.txt and parse each line as a requirement
    with open(requirements_path, 'r') as f:
        requirements = [line.strip() for line in f if line.strip()]

    # Fetch the list of currently installed packages
    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # Validate each requirement
    all_requirements_met = True
    for requirement in requirements:
        # Parse the requirement string to Requirement object
        req = pkg_resources.Requirement.parse(requirement)

        # Check if the package is installed
        if req.key not in installed_packages:
            print(f"Package '{req}' is not installed.")
            all_requirements_met = False
            continue

        # Check if installed package meets the version requirement
        installed_version = installed_packages[req.key]
        if installed_version not in req:
            print(f"Installed version {installed_version} of '{req.key}' does not meet requirement '{req}'.")
            all_requirements_met = False

    return all_requirements_met


if validate_requirements('./common/requirements.txt'):
    print("All requirements are met.")
else:
    print("Some requirements are not met.")

All requirements are met.


In [ ]:
# Uncomment this line if you want to check the packages currently installed
# !pip freeze

### Initialize Resource Management Client

Accessing the Azure Resource Management Client will be needed in further validation steps.

See documentation at https://learn.microsoft.com/en-us/python/api/overview/azure/identity-readme?view=azure-python

In [5]:
# Initialize Azure credentials
from azure.identity import AzureCliCredential
credential = AzureCliCredential()

# Initialize a Workspace object from the existing workspace you have
ws = Workspace.from_config()

# Retrieve and print the subscription ID
subscription_id = ws.subscription_id
print("Subscription ID:", subscription_id)

# Initialize the Resource Management client
resource_client = ResourceManagementClient(credential, subscription_id)

Subscription ID: 22d888ba-fc6d-4539-8483-172985f2a28f


### Validation:  Azure Cognitive Search

Check whether the credentials.env settings for Azure Cognitive Search lead to an active session.

In [6]:
def check_azure_search_service(api_version, endpoint, search_key):
    headers = {
        'api-key': search_key,
        'Content-Type': 'application/json'
    }
    params = {'api-version': api_version}

    # Construct the URL for the request
    url = f"{endpoint}/servicestats"

    try:
        # Make the API request
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            print("Azure Cognitive Search service is active.  The following output shows the service statistics.")
        else:
            print(f"Received status code {response.status_code}. Service may not be active or the request was invalid.")

        # Parse the JSON content
        json_data = json.loads(response.content)

        # Pretty-print the JSON data
        pretty_str = json.dumps(json_data, indent=4)
        print(pretty_str)

    except Exception as e:
        print(f"An error occurred: {e}")

# Set your own API version, endpoint, and search key
api_version = os.environ['AZURE_SEARCH_API_VERSION']
endpoint = os.environ['AZURE_SEARCH_ENDPOINT']
search_key = os.environ['AZURE_SEARCH_KEY']

check_azure_search_service(api_version, endpoint, search_key)

Azure Cognitive Search service is active.  The following output shows the service statistics.
{
    "@odata.context": "https://cog-search-rylu5pcprg6ja.search.azure.us/$metadata#Microsoft.Azure.Search.V2023_07_01_Preview.ServiceStatistics",
    "counters": {
        "documentCount": {
            "usage": 103813,
            "quota": null
        },
        "indexesCount": {
            "usage": 6,
            "quota": 50
        },
        "indexersCount": {
            "usage": 5,
            "quota": 50
        },
        "dataSourcesCount": {
            "usage": 5,
            "quota": 50
        },
        "storageSize": {
            "usage": 1337839012,
            "quota": 26843545600
        },
        "synonymMaps": {
            "usage": 0,
            "quota": 5
        },
        "skillsetCount": {
            "usage": 5,
            "quota": 50
        },
        "aliasesCount": {
            "usage": 0,
            "quota": 100
        },
        "vectorIndexSize": {
  

#### Semantic Search

Semantic search for Azure Cognitive Search may not be available for your region, and may not be enabled for your search service.
The best way to check is to use the Azure Portal using instructions from https://learn.microsoft.com/en-us/azure/search/semantic-how-to-enable-disable?tabs=enable-portal#enable-semantic-search

#### 